In [1]:
# docker run -p 8888:8888 jupyter/datascience-notebook:latest
import os
import datetime
import numpy as np
import pandas as pd

import qgrid
qgrid.enable()
# qgrid.disable()
import pandas_bokeh
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')

import mlfinlab as ml

import api
import bars

%load_ext autoreload
%autoreload 2

# import ipdb; ipdb.set_trace(context=12)
# import pudb; pu.db

Loading BokehJS ...

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# api.get_trades_dates(symbol, start_date=date, end_date=date, save='both')

In [4]:
symbol='GLD'
# symbol='SPY'
fmat = 'parquet'
date = '2019-05-09'

ts = api.read_matching_files(f"../data/ticks/{fmat}/{symbol}/{symbol}_{date}.{fmat}", format=fmat)
ts = bars.epoch_to_dt(ts)
ts = ts.set_index('date_time')

In [6]:
bars_v, _ = bars.build_bars(ts, thresh={'volume': 50000}, as_df=True)

dropping outlier tick:  -0.0036655683690280673
{'epoch': 1557415759113298000, 'price': 120.955, 'volume': 52966}
dropping outlier tick:  0.0036790541937084652
{'epoch': 1557415760302363000, 'price': 121.4, 'volume': 100}
dropping zero volume tick
{'epoch': 1557432600010354000, 'price': 121.2, 'volume': 0}
dropping zero volume tick
{'epoch': 1557441000010830000, 'price': 121.2, 'volume': 0}
dropping outlier tick:  0.0020635575732562937
{'epoch': 1557445557153388000, 'price': 121.4, 'volume': 5}
dropping zero volume tick
{'epoch': 1557446400012163000, 'price': 121.2, 'volume': 0}
dropping outlier tick:  -0.002387027738908487
{'epoch': 1557446400012262000, 'price': 121.2, 'volume': 50263}


In [10]:
figsize = (800, 450)
ts.price.plot_bokeh(rangetool=True, figsize=figsize)

Column(id='1094', ...)

In [12]:
bars_v.price_vwap.plot_bokeh(rangetool=True, figsize=(1000, 600), kind='line')`

Column(id='1761', ...)

In [11]:
bars_v.duration_min.plot_bokeh(rangetool=False, figsize=(1000, 600), kind='hist', bins=300, show_average=True)

Figure(id='1356', ...)

In [13]:
cols = ['duration_min', 'price_vwap', 'tick_count', 'volume_sum', 'tick_run_max', 'volume_imbalance']
bars_v[cols]

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [40]:
tsi.price.plot_bokeh(figsize=(1000, 600))

Figure(id='2693', ...)

In [37]:
def get_ticks(ts, start_at, trade_length='1 hour'):
    end_at = start_at + pd.to_timedelta(trade_length)
    return ts[start_at:end_at]
    

In [38]:
tsi = get_ticks(ts, start_at=bars_v.index[3])


In [50]:
tsi.price.iloc[2]

121.02

In [ ]:
def get_labels(ts, duration, rrr=3, pct_change_unit=0.001):
    
    buy_target = start_price + pct_change_unit * rrr
    buy_stop = start_price - pct_change_unit
    
    sell_target = start_price - pct_change_unit * rrr
    sell_stop = start_price + pct_change_unit
    
    for t in len(ts):
        if ts.price.iloc[t] < lower_thresh:
        
        elif ts.price.iloc[t] > upper_thresh:


In [ ]:
from bokeh.io import show, output_notebook
output_notebook()
from bokeh.layouts import gridplot
from bokeh.plotting import figure

x = bars_v.first_tick_at
y0 = bars_v.price_vwap
# y1 = bars_v.volume_sum
y1 = bars_v.volume_sum / (bars_v.price_vwap * 1.3)
y2 = bars_v.tick_imbalance

# create a new plot
s1 = figure(plot_width=800, plot_height=250, title=None)
s1.line(x, y0, line_width=2, color="navy", alpha=0.5)

# create a new plot and share both ranges
s2 = figure(plot_width=800, plot_height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.vbar(x, top=y1, width=2, color="firebrick", alpha=0.5)

# create a new plot and share only one range
# s3 = figure(plot_width=800, plot_height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
# s3.vbar(x, top=y2, width=2, color="olive", alpha=0.5)

p = gridplot([[s1], [s2]])

# show the results`
show(p)